In [4]:
# Import redirect csv to pandas dataframe
import pandas as pd
#Import selenium stuff
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

In [5]:
#General information about the data
initialRank = 1
finalRank = 30
year = "2022"
surface = "clay"

#Information of player 1
player1 = "Novak Djokovic"

#Information of player 2
player2 = "Rafael Nadal"

In [6]:
# This portion of code is used to obtain the code for each player
# ATPtour uses an individual code for each player. This is used in the url, next to the name of the player
#Select the ranking information 'initialRank' to 'finalRank'
driver = webdriver.Chrome('./chromedriver.exe')
driver.get("https://www.atptour.com/en/rankings/singles?rankRange="+str(initialRank)+"-"+str(finalRank))
data_table = []
list_href_players = []
code_for_players = []

for rank in range(initialRank, finalRank+1):
    link = driver.find_element_by_xpath("//*[@id='player-rank-detail-ajax']/tbody/tr["+str(rank)+"]/td[4]/span/a")
    list_href_players.append(link.get_attribute('href'))
    code = link.get_attribute('href')[-13:-9]
    code_for_players.append(code)

for table in driver.find_elements_by_xpath('//*[contains(@id,"player-rank-detail-ajax")]//tr'):
    data = [item.text for item in table.find_elements_by_xpath(".//*[self::td or self::th]")]
    print(data)
    data_table.append(data)
driver.close()

df_data = pd.DataFrame(data_table)
headers = df_data.iloc[0].values
df_data.columns = headers
df_data.drop(index=0, axis=0, inplace=True)
df_data['Code'] = code_for_players
df_data = df_data.drop('+/- Points', axis=1)
df_data = df_data.drop('Dropping', axis=1)
df_data = df_data.drop('Next Best', axis=1)

# Export pandas DataFrame as CSV
df_data.to_csv("./csv/ranking_01_23_23-"+str(initialRank)+"-"+str(finalRank)+"_"+ surface +".csv", index = True)          

#Code to read the csv file instead of gathering the information online
df = pd.read_csv("./csv/ranking_01_23_23-"+str(initialRank)+"-"+str(finalRank)+"_" + surface +".csv") 

df.fillna(0, inplace=True)
df.head()

['Rank', '+/- Rank', '', 'Player', 'Age', 'Points', '+/- Points', 'Tourn Played', 'Dropping', 'Next Best']
['1', '', '', 'Novak Djokovic', '35', '7,160', '', '16', '10', '0']
['2', '', '', 'Carlos Alcaraz', '19', '6,780', '', '18', '10', '0']
['3', '', '', 'Stefanos Tsitsipas', '24', '5,770', '', '21', '1,000', '45']
['4', '+1', '', 'Casper Ruud', '24', '5,255', '+250', '25', '90', '0']
['5', '-1', '', 'Daniil Medvedev', '27', '5,150', '', '23', '0', '0']
['6', '', '', 'Andrey Rublev', '25', '3,470', '', '25', '90', '45']
['7', '', '', 'Felix Auger-Aliassime', '22', '3,450', '', '26', '0', '0']
['8', '+1', '', 'Jannik Sinner', '21', '3,345', '', '20', '180', '0']
['9', '-1', '', 'Holger Rune', '19', '3,335', '-35', '26', '70', '20']
['10', '', '', 'Taylor Fritz', '25', '3,065', '', '22', '180', '0']
['11', '+4', '', 'Frances Tiafoe', '25', '2,870', '+205', '23', '0', '0']
['12', '-1', '', 'Karen Khachanov', '26', '2,855', '', '23', '0', '0']
['13', '-1', '', 'Hubert Hurkacz', '26', '2,

,Unnamed: 0,Rank,+/- Rank,Unnamed: 3,Player,Age,Points,Tourn Played,Code
0,1,1,0.0,0.0,Novak Djokovic,35,"7,160",16,d643
1,2,2,0.0,0.0,Carlos Alcaraz,19,"6,780",18,a0e2
2,3,3,0.0,0.0,Stefanos Tsitsipas,24,"5,770",21,te51
3,4,4,1.0,0.0,Casper Ruud,24,"5,255",25,rh16
4,5,5,-1.0,0.0,Daniil Medvedev,27,"5,150",23,mm58


In [36]:
def code_for_player(name, dataframe):
    i, c = np.where(dataframe == name)
    rowNumber = dataframe.index[i][0]
    code = dataframe['Code'][rowNumber]
    return code
#Creates a class to check the statistics of a player
class Player:
    def __init__(self, name, code, year, surface):
        self.name = name
        self.code = code
        self.year = year
        self.surface = surface        
        
    def getInformation(self):
        driver = webdriver.Chrome('./chromedriver.exe')
        driver.get("https://www.atptour.com/en/players/"+ self.name +"/"+self.code +"/player-stats?year="+str(self.year)+"&surfaceType="+self.surface)
        # Retrieve the current ranking in ATP tour 
        self.currentRanking = driver.find_element_by_xpath( "//*[@id='playerProfileHero']/div[2]/div[1]/div/div[3]/div[1]/div[2]").text
        # Aces 
        self.aces = driver.find_element_by_xpath( "//*[@id='playerMatchFactsContainer']/table[1]/tbody/tr[1]/td[2]").text
        # Total number of service games 
        self.serviceGamesPlayed = driver.find_element_by_xpath("//*[@id='playerMatchFactsContainer']/table[1]/tbody/tr[8]/td[2]").text
        # Double Fault
        self.doubleFaults = driver.find_element_by_xpath("//*[@id='playerMatchFactsContainer']/table[1]/tbody/tr[2]/td[2]").text
        # Percentage of break points in service games
        self.breakPointsSaved = driver.find_element_by_xpath("//*[@id='playerMatchFactsContainer']/table[1]/tbody/tr[7]/td[2]").text[0:2]
        # Percentage of wins in service games
        self.serviceGamesWonPercentage = driver.find_element_by_xpath("//*[@id='playerMatchFactsContainer']/table[1]/tbody/tr[9]/td[2]").text[0:2]
        # Percentage of breaking services of the opponent
        self.breakPointsConverted = driver.find_element_by_xpath("//*[@id='playerMatchFactsContainer']/table[2]/tbody/tr[4]/td[2]").text[0:2]
        # Percentage of games won by breaking the service 
        self.returnGamesWon = driver.find_element_by_xpath("//*[@id='playerMatchFactsContainer']/table[2]/tbody/tr[6]/td[2]").text[0:2]
        self.averageAcesPerGame = int(self.aces)/ int(self.serviceGamesPlayed)
        self.averageDoubleFaultsPerGame = int(self.doubleFaults)/ int(self.serviceGamesPlayed)
#Ranking position change
        
#1. get career high and date
#2. Best Ranking and when it happened
       #driver.get("https://www.atptour.com/en/players/"+ self.name +"/"+self.code +"/player-stats?year="+str(self.year)+"&surfaceType="+self.surface)


        
        
        driver.close()
        
#    def calculateAveragesPerGames(self,totalGames,aces,doubleFaults):

def head2head(player1code,player2code):
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.get("https://www.atptour.com/en/players/atp-head-2-head/one-vs-two/"+ player1code +"/" + player2code)
    player1Wins = driver.find_element_by_xpath("//*[@id='pageScoresH2h']/div[3]/div[1]/div[2]/div/div[1]").text
    player2Wins = driver.find_element_by_xpath("//*[@id='pageScoresH2h']/div[3]/div[3]/div[2]/div/div[1]").text
    driver.close()
    return player1Wins,player2Wins

In [38]:
#Objects called players to get the relevant information by using the general information commented at the beggining
Player1 = Player(player1, code_for_player(player1, df) , year, surface)
Player2 = Player(player2, code_for_player(player2, df) , year, surface)

# Call the getInformation method
Player1.getInformation()
Player2.getInformation()

# Call to the head2head funtction to compare if both players have played before and 
headToHead = head2head(Player1.code,Player2.code)


player1Wons = headToHead[0]
player2Wons = headToHead[1]




player1Stats has the following information:

- Ranking position (currentRanking)
- Aces (aces)
- Total of service games played (serviceGamesPlayed)
- Number of double faults (doubleFaults)
- Percentage of Aces per game (averageAcesPerGame)
- Percentage of Double Faults per game (averageDoubleFaultsPerGame)
- Number of break points saved (breakPointsSaved)
- Percentage of win a service game (serviceGamesWonPercentage)
- Percentage of break points converted (breakPointsConverted)
- Percentage of return games won (returnGamesWon)

In [41]:
print("Player 1 = "+ player1)
print("Ranking position = " + str(Player1.currentRanking))
print("Aces = " +  str(Player1.serviceGamesPlayed))
print("Total of service games played = " + str(Player1.aces))
print("Number of double faults = " + str(Player1.doubleFaults))
print("Number of double faults = " + str(Player1.averageAcesPerGame))
print("Percentage of Double Faults per game = " + str(Player1.averageDoubleFaultsPerGame))
print("Number of break points saved = " + str(Player1.breakPointsSaved))
print("Percentage of win a service game = " + str(Player1.serviceGamesWonPercentage))
print("Percentage of break points converted = " + str(Player1.breakPointsConverted))
print("Percentage of return games won = " + str(Player1.returnGamesWon))








Player 1 = Novak Djokovic
Ranking position = 1
Aces = 226
Total of service games played = 86
Number of double faults = 34
Number of double faults = 0.3805309734513274
Percentage of Double Faults per game = 0.1504424778761062
Number of break points saved = 66
Percentage of win a service game = 82
Percentage of break points converted = 41
Percentage of return games won = 31
Player 1 = Rafael Nadal
15
165
28
29
0.1696969696969697
0.17575757575757575
51
79
44
40


In [ ]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get("https://www.atptour.com/en/players/""/"+self.code +"/player-stats?year="+str(self.year)+"&surfaceType="+self.surface)
     

In [ ]:
1. El nombre tiene que ser separado por "_" en vez de espacio. 
2. El codigo por jugador es utilizado
3. Es necesario obtener la cantidad de victorias y derrotas